In [1]:
import pandas as pd
import numpy as np
from wikimapper import WikiMapper

# Load the wikidata index 
The file has to be downloaded using the following [link](https://public.ukp.informatik.tu-darmstadt.de/wikimapper/).

In [2]:
mapper = WikiMapper("data/index_enwiki-latest.db")

## Load CMU Movie Dataset

In [3]:
# Define columns names
cmu_movie_cols = ['wikipedia_id', 'freebase_id', 'cmu_movie_title', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres']

# Open movie dataset and remove 'freebase_id' since it is not available anymore
# Add movie wikidata ID using the mapper
cmu_movie_df = (pd.read_csv('data/movie.metadata.tsv', sep='\t', header=None, names=cmu_movie_cols)
                .drop(['freebase_id'], axis=1)
                .assign(
                    wikidata_id = lambda x: x.wikipedia_id.apply(lambda y: mapper.wikipedia_id_to_id(y))
                )
)

## Find movies based on books / book series
To find all movies that are based on books we run the following SPARQL query on the [Wikidata Query Service](https://query.wikidata.org/):

<code>
SELECT DISTINCT ?movie ?book
  
WHERE 
{

  VALUES ?bookType { wd:Q47461344 wd:Q7725634 wd:Q571 wd:Q14406742 wd:Q21198342 wd:Q277759}

  VALUES ?movieType { wd:Q11424 wd:Q506240 }
  
  ?book wdt:P31 ?bookType.   

  ?movie wdt:P31 ?movieType;          
        
        wdt:P144 ?book.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
</code>

It searches for instances of `film` or `television film` that are based on an instance of `literary work`, `written work`, `book`, `comic book series`, `manga series` or `book series`. This query gives a csv file which can be found in `data/raw_wiki/raw_movie_book.csv`. 

In [4]:
# Open the csv file we just created and extract the wikidata ID of movies and their corresponding book.
movie_book_df = (pd.read_csv('data/raw_wiki/raw_movie_book.csv')
                    .assign(
                        movie_wikidata_id = lambda x: x.movie.str.split('/').str[-1],
                        book_wikidata_id = lambda x: x.book.str.split('/').str[-1],
                    )
                    .drop(['movie', 'book'], axis=1)
                )

In [5]:
# Merge the CMU dataframe with the movie_book_df containing the wikidata ID
movie_book_df = (movie_book_df.merge(cmu_movie_df, left_on='movie_wikidata_id', right_on='wikidata_id', how='left')
              .query('wikipedia_id.notnull()')
              .reset_index(drop=True)
              .assign(movie_wikipedia_id = lambda x: x.wikipedia_id.astype(int))
              .loc[:, ['movie_wikipedia_id', 'book_wikidata_id']]
            )

In [6]:
display(movie_book_df)

,movie_wikipedia_id,book_wikidata_id
0,18920019,Q480
1,21447227,Q480
2,2205704,Q480
3,7379134,Q480
4,10117133,Q480
...,...,...
4677,9767560,Q120669834
4678,1750951,Q123168810
4679,61191,Q121775426
4680,6851697,Q122186265


# Clean CMU movies dataset

In [7]:
# Open the CMU dataset
movie_df = pd.read_csv("data/movie.metadata.tsv", sep='\t',names=['movie_wikipedia_id', 'freebase_id', 'cmu_movie_title', 'movie_release_date', 
                                                                  'movie_box_office_revenue', 'runtime', 'movie_languages', 'movie_countries', 
                                                                  'movie_genres']).drop('freebase_id', axis=1).assign(
                    movie_wikidata_id = lambda x: x.movie_wikipedia_id.apply(lambda y: mapper.wikipedia_id_to_id(y))
                )

#Clean columns
movie_df['movie_genres'] = movie_df['movie_genres'].apply(lambda x: np.take(x.split('"'), np.linspace(3,len(x.split('"'))-2, int((len(x.split('"'))-1)/4)).tolist()))
movie_df['movie_languages'] = movie_df['movie_languages'].apply(lambda x: np.take(x.split('"'), np.linspace(3,len(x.split('"'))-2, int((len(x.split('"'))-1)/4)).tolist()))
movie_df['movie_countries'] = movie_df['movie_countries'].apply(lambda x: np.take(x.split('"'), np.linspace(3,len(x.split('"'))-2, int((len(x.split('"'))-1)/4)).tolist()))
display(movie_df)

,movie_wikipedia_id,cmu_movie_title,movie_release_date,movie_box_office_revenue,runtime,movie_languages,movie_countries,movie_genres,movie_wikidata_id
0,975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",Q261700
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",Q16250726
2,28463795,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",Q4978832
3,9363483,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",Q7995657
4,261236,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama],Q869644
...,...,...,...,...,...,...,...,...,...
81736,35228177,Mermaids: The Body Found,2011-03-19,NaN,120.0,[English Language],[United States of America],[Drama],Q6819873
81737,34980460,Knuckle,2011-01-21,NaN,96.0,[English Language],"[Ireland, United Kingdom]","[Biographical film, Drama, Documentary]",Q12125420
81738,9971909,Another Nice Mess,1972-09-22,NaN,66.0,[English Language],[United States of America],"[Satire, Comedy]",Q4770308
81739,913762,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,[Japanese Language],[Japan],"[Science Fiction, Japanese Movies, Adventure, ...",Q2663931


# Add movies ratings to CMU movies dataset

To add movies ratings from IMDB to the CMU dataset, use the following query on [Wikidata Query Service](https://query.wikidata.org/).
First, we look for the IMDB ID of the movies.  

<code>

SELECT

?movie 

?IMDB_ID 

WHERE 

{

  VALUES ?movieType { wd:Q11424 wd:Q506240 }
  
  ?movie wdt:P31 ?movieType.

  ?movie wdt:P345 ?IMDB_ID.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  
}

</code>

This query gives a csv file containing IMDB ID and wikidata ID, which can be found in `data/raw_wiki/IMDb_id.csv`.

The non-commercial IMDb rating data set can be found [here](https://developer.imdb.com/non-commercial-datasets/). It contains the movie ratings (score out of 10) with the corresponding IMDb_ID. This dataset is in `data/IMDb_ratings.tsv`.

In [8]:
# Open IMDb_ratings file and clean the ID (remove 'tt' at the beginning)
ratings_df = pd.read_csv('data/IMDb_ratings.tsv', sep='\t').rename(columns={"tconst" : "imdb_id", "averageRating" : "movie_rating"})
ratings_df['imdb_id'] = ratings_df['imdb_id'].str[2:]
ratings_df.head()

,imdb_id,movie_rating,numVotes
0,0000001,5.7,2004
1,0000002,5.8,269
2,0000003,6.5,1902
3,0000004,5.5,178
4,0000005,6.2,2685


Now, let's add the movie ratings to the movie dataframe :

In [9]:
# Open IMDb_id and merge with ratings
IMDb_ID_df = pd.read_csv('data/raw_wiki/IMDb_id.csv').assign(wikidata_id = lambda x: x.movie.str.split('/').str[-1]).assign(imdb_id = lambda x: x['IMDB_ID'].str[2:]).drop(['movie', 'IMDB_ID'], axis=1)
IMDb_ID_df = IMDb_ID_df.merge(ratings_df, on='imdb_id', how='left').copy()
IMDb_ID_df.head(5)

# Merge rating to movie_df using the wikidata_id
final_movie_df = movie_df.merge(IMDb_ID_df, left_on='movie_wikidata_id', right_on='wikidata_id', how='left').drop('wikidata_id', axis=1)
display(final_movie_df)

,movie_wikipedia_id,cmu_movie_title,movie_release_date,movie_box_office_revenue,runtime,movie_languages,movie_countries,movie_genres,movie_wikidata_id,imdb_id,movie_rating,numVotes
0,975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",Q261700,0228333,4.9,56854.0
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",Q16250726,0245916,6.0,69.0
2,28463795,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",Q4978832,0094806,5.6,40.0
3,9363483,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",Q7995657,0094320,6.1,2888.0
4,261236,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama],Q869644,0083949,6.0,621.0
...,...,...,...,...,...,...,...,...,...,...,...,...
81906,35228177,Mermaids: The Body Found,2011-03-19,NaN,120.0,[English Language],[United States of America],[Drama],Q6819873,1816585,4.6,1710.0
81907,34980460,Knuckle,2011-01-21,NaN,96.0,[English Language],"[Ireland, United Kingdom]","[Biographical film, Drama, Documentary]",Q12125420,1606259,6.8,3191.0
81908,9971909,Another Nice Mess,1972-09-22,NaN,66.0,[English Language],[United States of America],"[Satire, Comedy]",Q4770308,0362411,5.8,110.0
81909,913762,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,[Japanese Language],[Japan],"[Science Fiction, Japanese Movies, Adventure, ...",Q2663931,NaN,NaN,NaN


Finally, let's modify the 'release_date' column to keep only the release year for each movie.

In [10]:
# Add a column with movie release year to the dataframe

# Keep only year in the string 'year-month-day'
final_movie_df['movie_release_year'] = final_movie_df['movie_release_date'].str.split('-').str[0].astype("Int32")

# Drop the 'movie_release_date' column
final_movie_df.drop('movie_release_date', axis=1)

,movie_wikipedia_id,cmu_movie_title,movie_box_office_revenue,runtime,movie_languages,movie_countries,movie_genres,movie_wikidata_id,imdb_id,movie_rating,numVotes,movie_release_year
0,975900,Ghosts of Mars,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",Q261700,0228333,4.9,56854.0,2001
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",Q16250726,0245916,6.0,69.0,2000
2,28463795,Brun bitter,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",Q4978832,0094806,5.6,40.0,1988
3,9363483,White Of The Eye,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",Q7995657,0094320,6.1,2888.0,1987
4,261236,A Woman in Flames,NaN,106.0,[German Language],[Germany],[Drama],Q869644,0083949,6.0,621.0,1983
...,...,...,...,...,...,...,...,...,...,...,...,...
81906,35228177,Mermaids: The Body Found,NaN,120.0,[English Language],[United States of America],[Drama],Q6819873,1816585,4.6,1710.0,2011
81907,34980460,Knuckle,NaN,96.0,[English Language],"[Ireland, United Kingdom]","[Biographical film, Drama, Documentary]",Q12125420,1606259,6.8,3191.0,2011
81908,9971909,Another Nice Mess,NaN,66.0,[English Language],[United States of America],"[Satire, Comedy]",Q4770308,0362411,5.8,110.0,1972
81909,913762,The Super Dimension Fortress Macross II: Lover...,NaN,150.0,[Japanese Language],[Japan],"[Science Fiction, Japanese Movies, Adventure, ...",Q2663931,NaN,NaN,NaN,1992


### The final movie dataframe is ready ! Let's save it for the analysis :

In [11]:
final_movie_df.to_csv('data/final_movie_metadata.csv')